#### Fast Mutual Exclusion

Let's first at ChatGPT to implement Lamport's fast mutual exclusion in Java.

In [1]:
import java.util.concurrent.atomic.AtomicInteger;

class LamportMutex {
    private final int numProcesses; // Total number of processes
    private AtomicInteger[] tickets; // Array to store ticket values
    private AtomicInteger[] entering; // Array to indicate process intentions

    public LamportMutex(int numProcesses) {
        this.numProcesses = numProcesses;
        tickets = new AtomicInteger[numProcesses];
        entering = new AtomicInteger[numProcesses];

        for (int i = 0; i < numProcesses; i++) {
            tickets[i] = new AtomicInteger();
            entering[i] = new AtomicInteger();
        }
    }

    public void lock(int processId) {
        entering[processId].set(1);
        tickets[processId].set(getMaxTicket() + 1);
        entering[processId].set(0);

        for (int i = 0; i < numProcesses; i++) {
            if (i != processId) {
                while (entering[i].get() == 1) {
                    // Wait until the other process finishes its ticket request
                }

                while (tickets[i].get() != 0 && (tickets[processId].get() > tickets[i].get() || (tickets[processId].get() == tickets[i].get() && processId > i))) {
                    // Wait until it's our turn to enter the critical section
                }
            }
        }
    }

    public void unlock(int processId) {
        tickets[processId].set(0);
    }

    private int getMaxTicket() {
        int maxTicket = Integer.MIN_VALUE;
        for (AtomicInteger ticket : tickets) {
            int ticketValue = ticket.get();
            if (ticketValue > maxTicket) {
                maxTicket = ticketValue;
            }
        }
        return maxTicket;
    }
}


int numProcesses = 5;
LamportMutex mutex = new LamportMutex(numProcesses);


for (int i = 0; i < numProcesses; i++) {
    final int processId = i;
    try {
    new Thread(() -> {
        try {
            Thread.sleep((long) (Math.random() * 1000));
            System.out.println("Process " + processId + " is trying to enter the critical section.");
            mutex.lock(processId);
            System.out.println("Process " + processId + " has entered the critical section.");
            Thread.sleep((long) (Math.random() * 1000));
            mutex.unlock(processId);
            System.out.println("Process " + processId + " has left the critical section.");
        } catch (InterruptedException e) {
             e.printStackTrace();
        }
    }).start();
    } catch (Exception e) {
        e.printStackTrace();
    }
}

Thread.sleep(20 * 1000);

System.out.println("Done");

Process 2 is trying to enter the critical section.
Process 2 has entered the critical section.
Process 0 is trying to enter the critical section.
Process 4 is trying to enter the critical section.
Process 0 has entered the critical section.
Process 2 has left the critical section.
Process 1 is trying to enter the critical section.
Process 3 is trying to enter the critical section.
Process 0 has left the critical section.
Process 4 has entered the critical section.
Process 1 has entered the critical section.
Process 4 has left the critical section.
Process 1 has left the critical section.
Process 3 has entered the critical section.
Process 3 has left the critical section.
Done


This implements mutual exclusion.  It is actually Lamport's [bakery algorithm](https://en.wikipedia.org/wiki/Lamport%27s_bakery_algorithm_), not fast mutual exclusion.  Watch out for our AI overlords leading us astray!
This is a cool algorithm that is:
* fair
* bounded-wait
* requires a linear amount of work in the number of processes.

I really don't like this example.  It is not a reasonable/efficient implementation of a mutual exclusion algorithms. Why not?

<!-- because it uses AtomicIntegers (already a synchronization construct) to implement a synchronization construct.  So, not good enough.  There are very simple direct algorithms that you can build on compare and swap primitives.
 This might be a fun in class exercise. -->

### Rewrite with volatile registers.

This is a skeleton of a Lamport FMEX implementation.  We are going to do a couple of  things differently.
  * we are going to use -1 to indicate no contention rather than zero.  This allows us to have a process 0.
  * Java doesn't allow arrays of volatile variables.  So, we have to create volatile variables and then build an array of references to them. I'm not positive that this works (BTW). I think it should, but it's a strange usage.
  
Fill in the TODOs.  I have added the relevant lines of pseudocode for Lamport's fast mutual exclusion algorithm.


In [5]:
public class LamportFMEX {
    
        // only works for 8
        private final int numProcesses;
        private boolean[] b = new boolean[8];
    
        // use -1 instead of 0 so we can have process 0
        private volatile int x=-1;
        private volatile int y=-1;

        // we need an array of volatile boolean variables
        private volatile boolean b0 = false;
        ...
        private volatile boolean b7 = false;

        public LamportFMEX (int numProcesses) {
            this.numProcesses = numProcesses;
            b[0] = this.b0;
            ...
            b[7] = this.b7;
        }

        public void lock(int processId) {
            // b[i] := true;
            b[processId] = true;
            // x := i
            x = processId;
            // if y != 0 then b[i] := false;
            //   await y = 0;
            //   goto start fi;
            if (y != 0 ) {
                b[processId] = false;
                while (y != 0) {
                    // wait
                }
                return; //go to start
            }
                
            // y := i;
            y = processId;
            //   if x != i then b[i] := false;
            //                  for j := 1 to n do await not b[j] od;
            //                  if y != i then await y = 0;
            //                      goto start fi fi;
            if (x != processId) {
                b[processId] = false;
                for (int j = 0; j < numProcesses; j++) {
                    while (b[j]) {
                        // wait
                    }
                }
                if (y != processId) {
                    while (y != 0) {
                        // wait
                    }
                    return; // go to start
                }
            }
            // critical section
            return;
        }
        public void unlock(int processId) {
            // y := 0
            y = 0;
            // b[i] := False
            b[processId] = false;
            return;
        }
}

In [6]:
LamportFMEX fmex = new LamportFMEX(2);

for (int i = 0; i < 8; i++) {
    final int processId = i;
    try {
        new Thread(() -> {
            try {
                Thread.sleep((long) (Math.random() * 1000));
                System.out.println("Process " + processId + " is trying to enter the critical section.");
                fmex.lock(processId);
                System.out.println("Process " + processId + " has entered the critical section.");
                Thread.sleep((long) (Math.random() * 1000));
                fmex.unlock(processId);
                System.out.println("Process " + processId + " has left the critical section.");
            } catch (InterruptedException e) {
                e.printStackTrace();
            }
        }).start();
    } catch (Exception e) {
        e.printStackTrace();
    }
}

Thread.sleep(20 * 1000);

Process 1 is trying to enter the critical section.
Process 0 is trying to enter the critical section.
Process 3 is trying to enter the critical section.
Process 5 is trying to enter the critical section.
Process 6 is trying to enter the critical section.
Process 7 is trying to enter the critical section.
Process 4 is trying to enter the critical section.
Process 2 is trying to enter the critical section.


### Debrief
